# YOUR PROJECT TITLE

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **data analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code from [lecture 5](https://numeconcopenhagen.netlify.com/lectures/Workflow_and_debugging).
> 1. Remember this [guide](https://www.markdownguide.org/basic-syntax/) on markdown and (a bit of) latex.
> 1. Turn on automatic numbering by clicking on the small icon on top of the table of contents in the left sidebar.
> 1. The `dataproject.py` file includes a function which will be used multiple times in this notebook.

Imports and set magics:

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from pandas_datareader import wb
import pandas_datareader
import pydst
import numpy as np
import json
import pickle
# Geopandas is used to load the world map data. 
import geopandas as gpd # Geopandas can be installed with conda install geopandas

# Bokeh is used for the interactive world map
from bokeh.io import output_notebook, show, output_file # Bokeh can be installed with conda install bokeh
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import brewer
from bokeh.io import curdoc, output_notebook
from bokeh.models import Slider, HoverTool
from bokeh.layouts import widgetbox, row, column



# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import dataproject

ValueError: source code string cannot contain null bytes

In [ ]:
Dst = pydst.Dst(lang="en") # Set DST language to english

In [ ]:
# We found a list of countrynames online at https://gist.github.com/pamelafox/986163 we then saved it using pickle and now open it again. We use this list later to only get countries when using the world bank api.
with open(f"dataproject.py", "rb") as f:
    data = pickle.load(f)
    countries = data["countries"]

# Pre-introduction code. The making of the world map

In [ ]:
worldshape = gpd.read_file("ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp")[["ADMIN", "ADM0_A3", "geometry"]] # We load the shp. file of the world map geometry. The source of the map is http://www.naturalearthdata.com/downloads/10m-cultural-vectors/, where we chose 1:110m scale Countries map, since this renders much quicker than more precise geometry (we learned that the hard way).

In [ ]:
worldshape.columns = ["country", "ISO3", "geometry"] # We change to column names
worldshape[4:5].iloc[0,0] = "United States"
worldshape[18:19].iloc[0,0] = "Russian Federation"
# We change the name of the United states and Russia to fit our list of data from World Data Bank. Preferably we'd like to use the ISO3 codes, as they are consisten, however we couldn't find a proper way to merge ISO3 codes onto the World Data Bank data.

In [ ]:
print(worldshape[worldshape['country'] == 'Antarctica']) # We locate the index for Antartica so we can remove it, because it takes up a lot of space on the map, but doesn't quite have an economy
worldshape = worldshape.drop(worldshape.index[159])

In [ ]:
# Using the previously mentioned countrylist, we create a new list that only has the ISO2 codes for the contries.
countrynames = []
for i in range(192):
    countrynames.append(countries[i]["code"])

In [ ]:
# We load the GDP growth data from the World Data Bank, using the countrynames list.
wb_gdp = wb.download(indicator='NY.GDP.MKTP.KD.ZG', country=countrynames, start=1990, end=2017)
wb_gdp = wb_gdp.reset_index() # Then make some formatting corrections, as the raw dataframe is encoded slightly weird
wb_gdp.year = wb_gdp.year.astype(int)

In [ ]:
wb_gdp.columns = ["country","year", "growth"] # We rename the columns

In [ ]:
# We define a function to take the GDP growth for a single year and merge it with the geometry file.
def split_merge(yr):
    g_year = wb_gdp[wb_gdp["year"] == yr] # We select only the data for the specified year
    g_year_merge = worldshape.merge(g_year, left_on = "country", right_on = "country", how = "left") 
    # And merge it with the geometry
    g_year_merge.loc[:,"growth"].fillna("No data", inplace = True) # Then we replace missing observations with "No data" - This is used later to color in missing observations
    return g_year_merge

In [ ]:
# Then we take 8 specfic years and create dataframes for them.
g2004merge = split_merge(2004)
g2005merge = split_merge(2005)
g2006merge = split_merge(2006)
g2007merge = split_merge(2007)
g2008merge = split_merge(2008)
g2009merge = split_merge(2009)
g2010merge = split_merge(2010)
g2011merge = split_merge(2011)

In [ ]:
# We define a function to convert the data to json format. This is because Bokeh reads json-format.
def convert_to_json(dataframe):
    dataframe_json = json.loads(dataframe.to_json())
    json_dataframe = json.dumps(dataframe_json)
    return json_dataframe

In [ ]:
# We then convert all the merged dataframes to json format. We could probably have used a loop, but annoyingly enough it didn't work, so we do it manually.

g2004merge_json = convert_to_json(g2004merge)
g2005merge_json = convert_to_json(g2005merge)
g2006merge_json = convert_to_json(g2006merge)
g2007merge_json = convert_to_json(g2007merge)
g2008merge_json = convert_to_json(g2008merge)
g2009merge_json = convert_to_json(g2009merge)
g2010merge_json = convert_to_json(g2010merge)
g2011merge_json = convert_to_json(g2011merge)

# I guess we should also explain why we do it individually for each year. We were going to make an interactive world map (and we did, as you can see later) but sadly it cannot run in the notebook (more on that when we get to the interactive map later), so instead we make a panel layout with several world maps you can tab between. 

In [ ]:
geosource = GeoJSONDataSource(geojson = g2008merge_json)
# Define a color palette, we've chosen red, yellow, green with 11 steps. For more see https://docs.bokeh.org/en/latest/docs/reference/palettes.html
palette = brewer["RdYlGn"][10]
# We then reverse the color palette so red indicates the lowest values of growth
palette = palette [::-1]

# Then we create the colormapper
color_mapper = LinearColorMapper(palette = palette, low = -5, high = 5, nan_color = '#d9d9d9')

# Define the labes for the color bar
tick_labels = {"-5": "<-5%", "-4": "-4%", "-3": "-3%", "-2": "-2%", "-1": "-1%", "0": "0%", "1": "1%", "2": "2%", "3": "3%", "4": "4%", "5": ">5%"}

# Then we create the actual color bar
color_bar = ColorBar(color_mapper=color_mapper,
                     label_standoff = 10, width = 500, height = 20,
                     border_line_color = None, location = (0,0), orientation = "horizontal",
                     major_label_overrides = tick_labels)
# WE create a hovertool, so it displays the exact growth when you hover over a country.
hover = HoverTool(tooltips = [("Country","@country"), ("% growth ", "@growth")])

# Create the figure
p = figure(title = "Annual GDP Growth, 2008", plot_height = 600, plot_width = 900, 
           toolbar_location = None, tools = [hover])
p.xgrid.grid_line_color = None 
p.ygrid.grid_line_color = None

# Add the patches for the figure - we add the geometry of the worldmap we earlier translated to GeoJson format
p.patches("xs","ys", source = geosource, fill_color = {"field":"growth", "transform": color_mapper}, line_color = "black", line_width = 0.25, fill_alpha = 1)

p.add_layout(color_bar, "below")
output_notebook()

In [ ]:
create_map_figure(g2004merge_json,g2004)

## Now we make it interactive

In [ ]:
# We define a function to return data for a given year:
def json_data(selectedYear):
    yr = selectedYear
    wb_gdp_yr = wb_gdp[wb_gdp["year"] == yr]
    merged = worldshape.merge(wb_gdp_yr, left_on = "country", right_on = "country", how = "left")
    merged.loc[:,"growth"].fillna("No data", inplace = True)
    merged_json = json.loads(merged.to_json())
    json_data = json.dumps(merged_json)
    return json_data

# The data source for plotting
geosource = GeoJSONDataSource(geojson = json_data(2008))

# We define the color palette, we choose red, yellow green
palette = brewer["RdYlGn"][10]
palette = palette[::-1] # We reverse color order so red means negative growth

# Then we create the colormapper using the palette, and defining the interval and the color for missing observations
color_mapper = LinearColorMapper(palette = palette, low = -5, high = 5, nan_color = '#d9d9d9')

# We define the tick labels for the color bar
tick_labels = {"-5": "<-5%", "-4": "-4%", "-3": "-3%", "-2": "-2%", "-1": "-1%", "0": "0%", "1": "1%", "2": "2%", "3": "3%", "4": "4%", "5": ">5%"}

# We add a HoverTool, so you can see the specific data when hovering over a country
hover = HoverTool(tooltips = [("Country","@country"), ("% growth ", "@growth")])

# We create the actual color bar
color_bar = ColorBar(color_mapper=color_mapper,
                     label_standoff = 10, width = 500, height = 20,
                     border_line_color = None, location = (0,0), orientation = "horizontal",
                     major_label_overrides = tick_labels)

# Then we create the figure objcet 
p = figure(title = "Annual GDP Growth, 2008", plot_height = 600, plot_width = 900, 
           toolbar_location = None, tools = [hover])
p.xgrid.grid_line_color = None 
p.ygrid.grid_line_color = None

# We then add the patches (geometry)
p.patches("xs","ys", source = geosource, fill_color = {"field":"growth", "transform": color_mapper}, line_color = "black", line_width = 0.25, fill_alpha = 1)

# And specify where the color bar goes
p.add_layout(color_bar, "below")

# Then we define a function to update the plot
def update_plot(attr, old, new):
    yr = slider.value
    new_data = json_data(yr)
    geosource.geojson = new_data
    p.title.text = "Annual percentage GDP growth"

# We then create the slider
slider = Slider(title = "Year", start = 1990, end = 2017, step = 1, value = 2009)
slider.on_change("value", update_plot)

# We then add create a column layout for the plot, so the slider is attached to the map
layout = column(p,widgetbox(slider))
curdoc().add_root(layout)

# We tell Bokeh that we want the output (the world map) to be shown in the notebook
output_notebook()

In [ ]:
show(layout)

## National accounts identity

In [ ]:
usvariables = ["GDPCA", "PCECCA","GPDICA", "IMPGSCA", "EXPGSCA"]
US = pandas_datareader.data.DataReader(usvariables,"fred", 1960,2019)
US.columns = ["GDP", "Private Consumption", "Private Investment", "Import", "Export"]
US.head(5)

In [ ]:
# Then we load employment data for USA, we use the 
US_EMP = pandas_datareader.data.DataReader("CE16OV","fred", 1960,2019)
US_EMP.columns = ["Employment"]

In [20]:
#Since data is monthly the annual average is calculated for each year 
a=0
b=[]
while a <= (708):
    mean = US_EMP.iloc[0+a:12+a,:].mean().round(0) # Dividing the columns into segments of 12 rows, before calculating average 
    b.append(mean) # the yearly means are saved in a list
    a+=12 #the loop moves 12 months 

Yearly_aggregate_mean=pd.DataFrame(b) #List turned into dataframe 

#"If it works it ain't stupid" - written in principia by Gandalf right after he blew up the Death Star, 2400 years before Mette Frederisken (the dear leader) ascended the Irone throne.

In [21]:
#US is given numerical index so the join functions can be used
US = US.reset_index()
#The two dataframes are joined together 
US_new = US.join(Yearly_aggregate_mean)


In [22]:
US_new.head(5)

,DATE,GDP,Private Consumption,Private Investment,Import,Export,Employment
0,1960-01-01,3259.971,2010.405,352.709,135.308,112.866,65784.0
1,1961-01-01,3343.546,2051.639,353.838,134.401,113.473,65744.0
2,1962-01-01,3548.409,2153.078,395.896,149.653,119.201,66702.0
3,1963-01-01,3702.944,2241.926,422.770,153.688,127.689,67760.0
4,1964-01-01,3916.280,2375.349,456.115,161.854,142.757,69301.0


In [23]:
#Index is set back to "DATE" 

US_new = US_new.set_index("DATE")

In [24]:
US_new = pd.melt(US_new.reset_index(), id_vars = US_new.index.name) #From wide to long - no idea why this works 


In [25]:
#The dataframe is sorted after date, and the index is reset 
US_new = US_new.sort_values("DATE") 
US_new=US_new.reset_index()
US_new=US_new.drop("index",axis=1)

In [26]:
US_new.tail(10)

,DATE,variable,value
350,2018-01-01,Employment,155760.000
351,2018-01-01,GDP,18638.164
352,2018-01-01,Import,3452.980
353,2018-01-01,Export,2532.938
354,2019-01-01,Private Investment,3421.330
355,2019-01-01,Import,3486.783
356,2019-01-01,Private Consumption,13280.072
357,2019-01-01,GDP,19073.056
358,2019-01-01,Export,2532.911
359,2019-01-01,Employment,156627.000


In [27]:
# We were originally going to make a table akin to TABLE 13.3a on page 371 in "Introducing Advanced Macroeconomics - Growth and Business Cycles", but way too late did we realize, we should have used quarterly data instead of annual. Luckily this is a course in programming rather than macro economics, so we stick to the plan and make the table with annual data.

# We begin creating the table, by calculating the correlation coefficient for between GDP and Private investmen, with annual lags of -2, -1, 0, 1 and 2 years.
neg2list = []
neg1list = []
nolaglist = []
pos1list = []
pos2list = []


for i in ["GDP", "Employment", "Import", "Private Investment", "Private Consumption", "Export"]:
    neg2 = np.corrcoef(US_new[US_new["variable"]=="GDP"]["value"].iloc[2:-2],US_new[US_new["variable"]== i ]       ["value"].iloc[:-4])[1][0]
    neg2list.append(neg2)

    neg1 = np.corrcoef(US_new[US_new["variable"]=="GDP"]["value"].iloc[2:-2],US_new[US_new["variable"]== i ]       ["value"].iloc[1:-3])[1][0]
    neg1list.append(neg1)
    
    nolag = np.corrcoef(US_new[US_new["variable"]=="GDP"]["value"].iloc[2:-2],US_new[US_new["variable"]== i ]       ["value"].iloc[2:-2])[1][0]

    nolaglist.append(nolag)

    pos1 = np.corrcoef(US_new[US_new["variable"]=="GDP"]["value"].iloc[2:-2],US_new[US_new["variable"]== i ]       ["value"].iloc[3:-1])[1][0]
    pos1list.append(pos1)

    pos2 = np.corrcoef(US_new[US_new["variable"]=="GDP"]["value"].iloc[2:-2],US_new[US_new["variable"]== i ]       ["value"].iloc[4:])[1][0]
    pos2list.append(pos2)

In [28]:
d = {"-2": neg2list, "-1": neg1list, "0": nolaglist, "1": pos1list, "2": pos2list}
dutufrume = pd.DataFrame(data=d)

dutufrumevars = {"Variable":["GDP", "Employment", "Import", "Private Investemt", "Private Consumption","Export"]}
dutufrumevarsdutu = pd.DataFrame(data = dutufrumevars)

dutufrume = dutufrume.join(dutufrumevarsdutu)
dutufrume = dutufrume.set_index("Variable")
dutufrume


,-2,-1,0,1,2
Variable,,,,,
GDP,0.997907,0.999226,1.000000,0.999237,0.997984
Employment,0.974488,0.973880,0.973493,0.971706,0.968715
Import,0.973882,0.979052,0.982801,0.983557,0.983950
Private Investemt,0.980937,0.983985,0.984876,0.980555,0.976010
Private Consumption,0.997021,0.998574,0.999453,0.999125,0.998258
Export,0.972670,0.976248,0.979900,0.981904,0.983758


In [29]:
US_new[US_new["variable"]=="GDP"]["value"].iloc[2:-2]

16      3548.409
19      3702.944
27      3916.280
35      4170.750
36      4445.853
44      4567.781
49      4792.315
55      4942.067
62      4951.262
66      5114.325
77      5383.282
81      5687.207
87      5656.465
90      5644.843
98      5948.995
106     6224.086
113     6568.608
116     6776.580
125     6759.181
131     6930.710
137     6805.758
142     7117.729
144     7632.812
151     7951.074
161     8226.392
166     8510.990
169     8866.498
177     9192.134
185     9365.494
188     9355.355
193     9684.892
199     9951.502
205    10352.432
210    10630.321
218    11031.350
223    11521.938
228    12038.283
236    12610.491
241    13130.987
247    13262.079
256    13493.064
260    13879.129
269    14406.382
273    14912.509
279    15338.257
283    15626.029
290    15604.687
295    15208.834
300    15598.753
311    15840.664
317    16197.007
323    16495.369
325    16912.038
331    17403.843
336    17688.890
345    18108.082
Name: value, dtype: float64

In [30]:
NAI_var = Dst.get_variables(table_id = "NAN1")
NAI_var
NAI_vars = Dst.get_variables(table_id='NAN1')
for id in ['TRANSAKT','PRISENHED']:
    print(id)
    values = NAI_vars.loc[NAI_vars.id == id,['values']].values[0,0]
    for value in values:      
        print(f' id = {value["id"]}, text = {value["text"]}')

TRANSAKT
 id = B1GQK, text = B.1*g Gross domestic product
 id = P7K, text = P.7 Imports of goods and services
 id = P71K, text = P.71 Import of goods
 id = P72K, text = P.72 Import of services
 id = TFSPR, text = Supply
 id = P6D, text = P.6 Exports of goods and services
 id = P61D, text = P.61 Export of goods
 id = P62D, text = P.62 Export of services
 id = P31S1MD, text = P.31 Private consumption
 id = P31S14D, text = P.31 Household consumption expenditure
 id = P311AD, text = Purchase of vehicles
 id = P311B_3D, text = Other goods
 id = P314A33S34D, text = Services incl. tourism
 id = P314D, text = Services
 id = P33D, text = Final consumption expenditure of resident households in the rest of the world
 id = P34D, text = Final consumption expenditure of non-resident households on the economic territory
 id = P31S15D, text = P.31 Non-profit institutions serving households (NPISH) consumption expenditure
 id = P3S13D, text = P.3 Government consumption expenditure
 id = P5GD, text = P.

In [31]:

variables = {"TRANSAKT":["B1GQK", "P7K", "P6D", "P31S1MD", "P5GD"], "PRISENHED":["LAN_M"], "TID":["*"]}
NAI = Dst.get_data(table_id = "NAN1", variables = variables)
NAI = NAI.drop("PRISENHED", axis = 1)

In [32]:
# Since employment data is can only be obtained for the price unit "Current prices" we have to get employment data seperately and then merge the two dataframes.
EMP = Dst.get_data(table_id = "NAN1", variables = {"TRANSAKT":["EMPM_DC"], "PRISENHED":["V_M"], "TID":["*"]})
EMP = EMP.drop("PRISENHED", axis = 1)

In [33]:
DKDATA = pd.merge(NAI, EMP, on = ["TRANSAKT", "TID","INDHOLD"], how = "outer")

In [34]:
DKDATA = DKDATA.sort_values("TID")
DKDATA.head(10)

,TRANSAKT,TID,INDHOLD
0,B.1*g Gross domestic product,1966,702.4
4,P.7 Imports of goods and services,1966,112.2
3,P.6 Exports of goods and services,1966,123.6
270,"Total employment (1,000 persons)",1966,2322.8
1,P.31 Private consumption,1966,372.8
2,P.5g Gross capital formation,1966,127.6
5,B.1*g Gross domestic product,1967,741.2
6,P.31 Private consumption,1967,397.9
7,P.5g Gross capital formation,1967,133.7
8,P.6 Exports of goods and services,1967,128.2


## Income data

In [35]:
g={"B.1*g Gross domestic product":"GDP","P.7 Imports of goods and services":"Import","P.6 Exports of goods and services":"Export","Total employment (1,000 persons)":"Employment","P.31 Private consumption":"Consumption","P.5g Gross capital formation":"Investment"}

## Explore data set

In [36]:
for key, value in g.items():
    DKDATA.TRANSAKT.replace(key,value,inplace=True)

# Merge data sets

In [37]:
DKDATA =DKDATA.reset_index()
DKDATA = DKDATA.drop("index", axis=1)

In [39]:
DKDATA.head(10)

,TRANSAKT,TID,INDHOLD
0,GDP,1966,702.4
1,Import,1966,112.2
2,Export,1966,123.6
3,Employment,1966,2322.8
4,Consumption,1966,372.8
5,Investment,1966,127.6
6,GDP,1967,741.2
7,Consumption,1967,397.9
8,Investment,1967,133.7
9,Export,1967,128.2


ADD FURTHER ANALYSIS. EXPLAIN THE CODE BRIEFLY AND SUMMARIZE THE RESULTS.

# Conclusion

ADD CONCISE CONLUSION.

In [40]:
my_data = {}
my_data["countries"] = countries
with open(f"dataproject.py", "wb") as f:
    pickle.dump(my_data,f)